In [ ]:
#!/usr/bin/env python3

import os
import sys
import RPi.GPIO as GPIO
import serial
import threading
import datetime
import time
import logging
import re
import tkinter as tk
from tkinter import ttk, filedialog
import queue
import csv

# Configure logging
logging.basicConfig(
    level=logging.INFO,
    format='%(asctime)s [%(levelname)s] %(message)s',
    stream=sys.stdout
)

# Setup GPIO pins on the Raspberry Pi (BCM mode)
GPIO.setmode(GPIO.BCM)
GPIO.setwarnings(False)

# Define GPIO pins for each device 
gpio_pins_per_device = {
    'Port 1': {"LeftPoke": 17, "RightPoke": 27, "Pellet": 22},
    'Port 2': {"LeftPoke": 10, "RightPoke": 9, "Pellet": 11},
    'Port 3': {"LeftPoke": 0, "RightPoke": 5, "Pellet": 6},
    'Port 4': {"LeftPoke": 13, "RightPoke": 19, "Pellet": 26},
}

# Set all pins as output and initially set them to LOW
for device_pins in gpio_pins_per_device.values():
    for pin in device_pins.values():
        GPIO.setup(pin, GPIO.OUT)
        GPIO.output(pin, GPIO.LOW)

# Define global threading and data storage variables
pellet_lock = threading.Lock()
pellet_in_well = {}
stop_event = threading.Event()
column_headers = [
    "Timestamp", "Temp", "Humidity", "Library_Version", "Session_type",
    "Device_Number", "Battery_Voltage", "Motor_Turns", "FR", "Event", "Active_Poke",
    "Left_Poke_Count", "Right_Poke_Count", "Pellet_Count", "Block_Pellet_Count",
    "Retrieval_Time", "InterPelletInterval", "Poke_Time"
]

def send_ttl_signal(pin):
    GPIO.output(pin, GPIO.HIGH)
    time.sleep(0.1)
    GPIO.output(pin, GPIO.LOW)

# Handle pellet events
def handle_pellet_event(event_type, port_identifier, gpio_pins, q):
    global pellet_in_well
    with pellet_lock:
        if port_identifier not in pellet_in_well:
            pellet_in_well[port_identifier] = False
        if event_type == "Pellet":
            if pellet_in_well[port_identifier]:
                GPIO.output(gpio_pins["Pellet"], GPIO.LOW)
                q.put(f"Pellet taken, signal turned OFF.")
                pellet_in_well[port_identifier] = False
                send_ttl_signal(gpio_pins["Pellet"])
            else:
                q.put(f"No pellet was in the well, no signal for pellet taken.")
        elif event_type == "PelletInWell":
            GPIO.output(gpio_pins["Pellet"], GPIO.HIGH)
            pellet_in_well[port_identifier] = True
            q.put(f"Pellet dispensed in well, signal ON.")

def process_event(event_type, port_identifier, gpio_pins, q):
    timestamp = datetime.datetime.now().strftime("%Y-%m-%d %H:%M:%S.%f")[:-3]
    message = f"[{timestamp}] {port_identifier} - Event: {event_type}"
    q.put(message)
    if event_type == "Left":
        send_ttl_signal(gpio_pins["LeftPoke"])
    elif event_type == "Right":
        send_ttl_signal(gpio_pins["RightPoke"])
    elif event_type == "LeftWithPellet":
        send_ttl_signal(gpio_pins["LeftPoke"])
    elif event_type == "RightWithPellet":
        send_ttl_signal(gpio_pins["RightPoke"])
    elif event_type in ["Pellet", "PelletInWell"]:
        handle_pellet_event(event_type, port_identifier, gpio_pins, q)

def get_device_mappings_by_usb_port():
    device_mappings = []
    usb_port_mapping = {'usb-0:1.1': 'Port 1', 'usb-0:1.2': 'Port 2', 'usb-0:1.3': 'Port 3', 'usb-0:1.4': 'Port 4'}
    for symlink in os.listdir('/dev/serial/by-path/'):
        symlink_path = os.path.join('/dev/serial/by-path/', symlink)
        serial_port = os.path.realpath(symlink_path)
        if 'ttyACM' in serial_port or 'ttyUSB' in serial_port:
            usb_port_path = get_usb_port_path_from_symlink(symlink)
            if usb_port_path in usb_port_mapping:
                device_mappings.append({'serial_port': serial_port, 'port_identifier': usb_port_mapping[usb_port_path]})
    return device_mappings

def get_usb_port_path_from_symlink(symlink):
    match = re.search(r'usb-\d+:\d+(\.\d+)*', symlink)
    return match.group() if match else None

def read_from_fed(serial_port, port_identifier, gpio_pins, q):
    try:
        ser = serial.Serial(serial_port, 115200, timeout=1)
        q.put("Ready")
        while not stop_event.is_set():
            line = ser.readline().decode('utf-8').strip()
            if line:
                data_list = line.split(",")
                q.put(f"{port_identifier} raw data: {data_list}")
                if len(data_list) >= 10:
                    event_type = data_list[9]
                    timestamp = datetime.datetime.now().strftime("%Y-%m-%d %H:%M:%S.%f")[:-3]
                    data_list[0] = timestamp
                    process_event(event_type, port_identifier, gpio_pins, q)
                    q.put(data_list)
    except serial.SerialException:
        q.put(f"Error opening serial port: {serial_port}")
    finally:
        ser.close()

class SplashScreen:
    def __init__(self, root, duration=3000):
        self.root = root
        self.root.overrideredirect(True)
        self.root.geometry("800x480")
        self.root.attributes("-fullscreen", True)

        self.root.configure(bg="black")
        self.label = tk.Label(self.root, text="McCutcheonlab Technologies", font=("Cascadia Code", 28, "bold"), bg="black", fg="lavender")
        self.label.pack(expand=True)
        self.root.after(duration, self.close_splash)

    def close_splash(self):
        self.root.destroy()

class FED3MonitorApp:
    def __init__(self, root):
        self.root = root
        self.root.title("HPFED TTL Monitor")
        self.root.geometry("800x480")
        self.root.attributes("-fullscreen", True)

        self.port_widgets = {}
        self.port_queues = {}
        self.experimenter_name = tk.StringVar()
        self.experiment_name = tk.StringVar()
        self.save_path = ""
        self.data_to_save = {}
        self.threads = []  # Initialize threads list

        # Mainframe layout
        self.mainframe = ttk.Frame(self.root)
        self.mainframe.grid(column=0, row=0, sticky=(tk.N, tk.W, tk.E, tk.S))
        self.root.grid_rowconfigure(0, weight=1)
        self.root.grid_columnconfigure(0, weight=1)

        # Layout Setup
        self.create_layout()
        self.check_connected_devices()
        self.root.after(100, self.update_gui)

    def create_layout(self):
        # Left side: Port 1 and Port 2
        left_ports = ttk.Frame(self.mainframe)
        left_ports.grid(column=0, row=0, sticky=(tk.N, tk.S, tk.W))

        self.setup_port(left_ports, 'Port 1', 0)
        self.setup_port(left_ports, 'Port 2', 1)

        # Right side: Port 3 and Port 4
        right_ports = ttk.Frame(self.mainframe)
        right_ports.grid(column=2, row=0, sticky=(tk.N, tk.S, tk.E))

        self.setup_port(right_ports, 'Port 3', 0)
        self.setup_port(right_ports, 'Port 4', 1)

        # Center controls
        controls_frame = ttk.Frame(self.mainframe)
        controls_frame.grid(column=1, row=0, padx=10, sticky=(tk.N, tk.S))

        tk.Label(controls_frame, text="Your Name:", font=("Cascadia Code", 8)).grid(column=0, row=0, sticky=tk.W)
        self.experimenter_entry = ttk.Entry(controls_frame, textvariable=self.experimenter_name, width=20)
        self.experimenter_entry.grid(column=1, row=0, sticky=tk.W)

        tk.Label(controls_frame, text="Experiment Name:", font=("Cascadia Code", 8)).grid(column=0, row=1, sticky=tk.W)
        self.experiment_entry = ttk.Entry(controls_frame, textvariable=self.experiment_name, width=20)
        self.experiment_entry.grid(column=1, row=1, sticky=tk.W)

        # Start and Stop buttons
        self.start_button = tk.Button(controls_frame, text="START", font=("Cascadia Code", 9, "bold"), bg="green", fg="white", command=self.start_experiment)
        self.start_button.grid(column=0, row=2, columnspan=2, sticky="we", pady=5)

        self.stop_button = tk.Button(controls_frame, text="STOP & SAVE", font=("Cascadia Code", 9, "bold"), bg="red", fg="white", command=self.stop_experiment)
        self.stop_button.grid(column=0, row=3, columnspan=2, sticky="we", pady=5)

        # Browse button
        self.browse_button = tk.Button(controls_frame, text="Browse Data Folder", font=("Cascadia Code", 9), command=self.browse_folder)
        self.browse_button.grid(column=0, row=4, columnspan=2, sticky="we", pady=5)

        # Recording Indicator
        self.canvas = tk.Canvas(controls_frame, width=120, height=100)
        self.canvas.grid(column=0, row=5, columnspan=2, pady=8)
        self.recording_circle = None  # Initialize as None
        self.recording_label = None   # Initialize as None

        # Footer copyright message
        self.mainframe.grid_rowconfigure(1, weight=1)
        copyright_label = tk.Label(self.mainframe, text="© 2024 McCutcheonlab | UiT | Norway", font=("Cascadia Code", 8), fg="black")
        copyright_label.grid(column=0, row=2, columnspan=3, sticky="s", pady=5)

    def setup_port(self, parent, port_name, row):
        frame = ttk.LabelFrame(parent, text=port_name, padding="3")
        frame.grid(column=0, row=row, padx=5, pady=5, sticky=(tk.N, tk.S, tk.W, tk.E))
        status_label = ttk.Label(frame, text="Not Ready", font=("Cascadia Code", 10), foreground="red")
        status_label.grid(column=0, row=0, sticky=tk.W)
        text_widget = tk.Text(frame, width=28, height=6, wrap=tk.WORD)
        text_widget.grid(column=0, row=1, sticky=(tk.N, tk.S, tk.E, tk.W))
        self.port_widgets[port_name] = {'status_label': status_label, 'text_widget': text_widget}
        self.port_queues[port_name] = queue.Queue()

    def browse_folder(self):
        self.save_path = filedialog.askdirectory(title="Select Folder to Save Data")

    def check_connected_devices(self):
        device_mappings = get_device_mappings_by_usb_port()
        for mapping in device_mappings:
            port_identifier = mapping['port_identifier']
            self.port_widgets[port_identifier]['status_label'].config(text="Ready", foreground="green")

    def display_recording_indicator(self):
        if self.recording_circle is None:
            self.recording_circle = self.canvas.create_oval(10, 10, 50, 50, fill="red")
        if self.recording_label is None:
            self.recording_label = self.canvas.create_text(39, 60, text="RECORDING", font=("Cascadia Code", 10),anchor= "n")

    def hide_recording_indicator(self):
        if self.recording_circle is not None:
            self.canvas.delete(self.recording_circle)
            self.recording_circle = None
        if self.recording_label is not None:
            self.canvas.delete(self.recording_label)
            self.recording_label = None

    def start_experiment(self):
        self.experimenter_entry.config(state='disabled')
        self.experiment_entry.config(state='disabled')
        
        self.experimenter_name.set(self.experimenter_name.get().strip().lower())
        self.experiment_name.set(self.experiment_name.get().strip().lower())
        self.data_to_save = {port_identifier: [] for port_identifier in self.port_widgets.keys()}
        self.threads = []  # Initialize threads list
        for mapping in get_device_mappings_by_usb_port():
            serial_port = mapping['serial_port']
            port_identifier = mapping['port_identifier']
            gpio_pins = gpio_pins_per_device.get(port_identifier)
            q = self.port_queues[port_identifier]
            t = threading.Thread(target=read_from_fed, args=(serial_port, port_identifier, gpio_pins, q))
            t.daemon = True
            t.start()
            self.threads.append(t)  # Store thread in self.threads
        self.display_recording_indicator()

    def update_gui(self):
        for port_identifier, q in self.port_queues.items():
            try:
                while True:
                    message = q.get_nowait()
                    if isinstance(message, list):
                        self.data_to_save[port_identifier].append(message)
                    elif message == "Ready":
                        self.port_widgets[port_identifier]['status_label'].config(text="Ready", foreground="green")
                    else:
                        text_widget = self.port_widgets[port_identifier]['text_widget']
                        text_widget.insert(tk.END, message + "\n")
                        text_widget.see(tk.END)
            except queue.Empty:
                pass
        self.root.after(100, self.update_gui)

    def stop_experiment(self):
        stop_event.set()
        for t in self.threads:
            t.join()
        GPIO.cleanup()
        self.save_all_data()
        self.hide_recording_indicator()
        self.root.quit()
        self.root.destroy()

    def save_all_data(self):
        current_time = datetime.datetime.now().strftime("%Y_%m_%d_%H_%M_%S")
        experimenter_folder = os.path.join(self.save_path, self.experimenter_name.get().strip())
        experiment_folder = os.path.join(experimenter_folder, f"{self.experiment_name.get().strip()}_{current_time}")
        os.makedirs(experiment_folder, exist_ok=True)
        for port_identifier, data_rows in self.data_to_save.items():
            # Save data in the experiment folder with folder structures
            filename_user = os.path.join(experiment_folder, f"{port_identifier}_{current_time}.csv")
            with open(filename_user, mode='w', newline='') as file:
                writer = csv.writer(file)
                writer.writerow(column_headers)
                writer.writerows(data_rows)

            # Save data directly into save_path without folder structures
            filename_main = os.path.join(self.save_path, f"{current_time}_{port_identifier}.csv")
            with open(filename_main, mode='w', newline='') as file:
                writer = csv.writer(file)
                writer.writerow(column_headers)
                writer.writerows(data_rows)

if __name__ == "__main__":
    splash_root = tk.Tk()
    splash_screen = SplashScreen(splash_root)
    splash_root.mainloop()

    root = tk.Tk()
    app = FED3MonitorApp(root)
    root.mainloop()


# Update

In [ ]:
#!/usr/bin/env python3

import os
import sys
import RPi.GPIO as GPIO
import serial
import threading
import datetime
import time
import logging
import re
import tkinter as tk
from tkinter import ttk, filedialog, messagebox
import queue
import csv

# Configure logging
logging.basicConfig(
    level=logging.INFO,
    format='%(asctime)s [%(levelname)s] %(message)s',
    stream=sys.stdout
)

# Setup GPIO pins on the Raspberry Pi (BCM mode)
GPIO.setmode(GPIO.BCM)
GPIO.setwarnings(False)

# Define GPIO pins for each device
gpio_pins_per_device = {
    'Port 1': {"LeftPoke": 17, "RightPoke": 27, "Pellet": 22},
    'Port 2': {"LeftPoke": 10, "RightPoke": 9, "Pellet": 11},
    'Port 3': {"LeftPoke": 0, "RightPoke": 5, "Pellet": 6},
    'Port 4': {"LeftPoke": 13, "RightPoke": 19, "Pellet": 26},
}

# Set all pins as output and initially set them to LOW
for device_pins in gpio_pins_per_device.values():
    for pin in device_pins.values():
        GPIO.setup(pin, GPIO.OUT)
        GPIO.output(pin, GPIO.LOW)

# Define global threading and data storage variables
pellet_lock = threading.Lock()
pellet_in_well = {}
stop_event = threading.Event()
column_headers = [
    "Timestamp", "Temp", "Humidity", "Library_Version", "Session_type",
    "Device_Number", "Battery_Voltage", "Motor_Turns", "FR", "Event", "Active_Poke",
    "Left_Poke_Count", "Right_Poke_Count", "Pellet_Count", "Block_Pellet_Count",
    "Retrieval_Time", "InterPelletInterval", "Poke_Time"
]

def send_ttl_signal(pin):
    GPIO.output(pin, GPIO.HIGH)
    time.sleep(0.1)
    GPIO.output(pin, GPIO.LOW)

# Handle pellet events
def handle_pellet_event(event_type, port_identifier, gpio_pins, q):
    global pellet_in_well
    with pellet_lock:
        if port_identifier not in pellet_in_well:
            pellet_in_well[port_identifier] = False
        if event_type == "Pellet":
            if pellet_in_well[port_identifier]:
                GPIO.output(gpio_pins["Pellet"], GPIO.LOW)
                q.put(f"Pellet taken, signal turned OFF.")
                pellet_in_well[port_identifier] = False
                send_ttl_signal(gpio_pins["Pellet"])
            else:
                q.put(f"No pellet was in the well, no signal for pellet taken.")
        elif event_type == "PelletInWell":
            GPIO.output(gpio_pins["Pellet"], GPIO.HIGH)
            pellet_in_well[port_identifier] = True
            q.put(f"Pellet dispensed in well, signal ON.")

def process_event(event_type, port_identifier, gpio_pins, q):
    timestamp = datetime.datetime.now().strftime("%Y-%m-%d %H:%M:%S.%f")[:-3]
    message = f"[{timestamp}] {port_identifier} - Event: {event_type}"
    q.put(message)
    if event_type == "Left":
        send_ttl_signal(gpio_pins["LeftPoke"])
    elif event_type == "Right":
        send_ttl_signal(gpio_pins["RightPoke"])
    elif event_type == "LeftWithPellet":
        send_ttl_signal(gpio_pins["LeftPoke"])
    elif event_type == "RightWithPellet":
        send_ttl_signal(gpio_pins["RightPoke"])
    elif event_type in ["Pellet", "PelletInWell"]:
        handle_pellet_event(event_type, port_identifier, gpio_pins, q)

def get_device_mappings_by_usb_port():
    device_mappings = []
    usb_port_mapping = {'usb-0:1.1': 'Port 1', 'usb-0:1.2': 'Port 2', 'usb-0:1.3': 'Port 3', 'usb-0:1.4': 'Port 4'}
    by_path_dir = '/dev/serial/by-path/'
    if not os.path.exists(by_path_dir):
        return device_mappings  # Return empty list if directory doesn't exist
    for symlink in os.listdir(by_path_dir):
        symlink_path = os.path.join(by_path_dir, symlink)
        serial_port = os.path.realpath(symlink_path)
        if 'ttyACM' in serial_port or 'ttyUSB' in serial_port:
            usb_port_path = get_usb_port_path_from_symlink(symlink)
            if usb_port_path in usb_port_mapping:
                device_mappings.append({'serial_port': serial_port, 'port_identifier': usb_port_mapping[usb_port_path]})
    return device_mappings

def get_usb_port_path_from_symlink(symlink):
    match = re.search(r'usb-\d+:\d+(\.\d+)*', symlink)
    return match.group() if match else None

def read_from_fed(serial_port, port_identifier, gpio_pins, q, status_label):
    try:
        ser = serial.Serial(serial_port, 115200, timeout=1)
        q.put("Ready")
        status_label.config(text="Connected", foreground="green")
        while not stop_event.is_set():
            try:
                line = ser.readline().decode('utf-8').strip()
                if line:
                    data_list = line.split(",")
                    q.put(f"{port_identifier} raw data: {data_list}")
                    if len(data_list) >= 10:
                        event_type = data_list[9]
                        timestamp = datetime.datetime.now().strftime("%Y-%m-%d %H:%M:%S.%f")[:-3]
                        data_list[0] = timestamp
                        process_event(event_type, port_identifier, gpio_pins, q)
                        q.put(data_list)
            except serial.SerialException:
                q.put(f"Device on {port_identifier} disconnected.")
                status_label.config(text="Disconnected", foreground="red")
                break  # Exit loop if device is disconnected
    except serial.SerialException:
        q.put(f"Error opening serial port: {serial_port}")
        status_label.config(text="Disconnected", foreground="red")
    finally:
        if 'ser' in locals() and ser.is_open:
            ser.close()
        q.put(f"Stopped reading from {port_identifier}")

class SplashScreen:
    def __init__(self, root, duration=3000):
        self.root = root
        self.root.overrideredirect(True)
        self.root.geometry("800x480")
        self.root.attributes("-fullscreen", True)

        self.root.configure(bg="black")
        self.label = tk.Label(self.root, text="McCutcheonlab Technologies", font=("Cascadia Code", 28, "bold"), bg="black", fg="lavender")
        self.label.pack(expand=True)
        self.root.after(duration, self.close_splash)

    def close_splash(self):
        self.root.destroy()

class FED3MonitorApp:
    def __init__(self, root):
        self.root = root
        self.root.title("HPFED TTL Monitor")
        self.root.geometry("800x480")
        self.root.attributes("-fullscreen", True)

        self.port_widgets = {}
        self.port_queues = {}
        self.experimenter_name = tk.StringVar()
        self.experiment_name = tk.StringVar()
        self.save_path = ""
        self.data_to_save = {}
        self.threads = []
        self.connected_ports = []
        self.serial_ports = {}  # Store serial objects
        self.logging_active = False

        # Mainframe layout
        self.mainframe = ttk.Frame(self.root)
        self.mainframe.grid(column=0, row=0, sticky=(tk.N, tk.W, tk.E, tk.S))
        self.root.grid_rowconfigure(0, weight=1)
        self.root.grid_columnconfigure(0, weight=1)

        # Layout Setup
        self.create_layout()
        self.check_connected_devices()
        self.root.after(100, self.update_gui)
        self.root.after(5000, self.refresh_device_status)  # Periodically refresh device status
        self.root.protocol("WM_DELETE_WINDOW", self.on_closing)

    def create_layout(self):
        # Left side: Port 1 and Port 2
        left_ports = ttk.Frame(self.mainframe)
        left_ports.grid(column=0, row=0, sticky=(tk.N, tk.S, tk.W))

        self.setup_port(left_ports, 'Port 1', 0)
        self.setup_port(left_ports, 'Port 2', 1)

        # Right side: Port 3 and Port 4
        right_ports = ttk.Frame(self.mainframe)
        right_ports.grid(column=2, row=0, sticky=(tk.N, tk.S, tk.E))

        self.setup_port(right_ports, 'Port 3', 0)
        self.setup_port(right_ports, 'Port 4', 1)

        # Center controls
        controls_frame = ttk.Frame(self.mainframe)
        controls_frame.grid(column=1, row=0, padx=10, sticky=(tk.N, tk.S))

        tk.Label(controls_frame, text="Your Name:", font=("Cascadia Code", 8)).grid(column=0, row=0, sticky=tk.W)
        self.experimenter_entry = ttk.Entry(controls_frame, textvariable=self.experimenter_name, width=20)
        self.experimenter_entry.grid(column=1, row=0, sticky=tk.W)

        tk.Label(controls_frame, text="Experiment Name:", font=("Cascadia Code", 8)).grid(column=0, row=1, sticky=tk.W)
        self.experiment_entry = ttk.Entry(controls_frame, textvariable=self.experiment_name, width=20)
        self.experiment_entry.grid(column=1, row=1, sticky=tk.W)

        # Start and Stop buttons
        self.start_button = tk.Button(controls_frame, text="START", font=("Cascadia Code", 9, "bold"), bg="green", fg="white", command=self.start_experiment)
        self.start_button.grid(column=0, row=2, columnspan=2, sticky="we", pady=5)

        self.stop_button = tk.Button(controls_frame, text="STOP & SAVE", font=("Cascadia Code", 9, "bold"), bg="red", fg="white", command=self.stop_experiment)
        self.stop_button.grid(column=0, row=3, columnspan=2, sticky="we", pady=5)

        # Browse button
        self.browse_button = tk.Button(controls_frame, text="Browse Data Folder", font=("Cascadia Code", 9), command=self.browse_folder)
        self.browse_button.grid(column=0, row=4, columnspan=2, sticky="we", pady=5)

        # Recording Indicator
        self.canvas = tk.Canvas(controls_frame, width=120, height=100)
        self.canvas.grid(column=0, row=5, columnspan=2, pady=8)
        self.recording_circle = None  # Initialize as None
        self.recording_label = None   # Initialize as None

        # Footer copyright message
        self.mainframe.grid_rowconfigure(1, weight=1)
        copyright_label = tk.Label(self.mainframe, text="© 2024 McCutcheonlab | UiT | Norway", font=("Cascadia Code", 8), fg="black")
        copyright_label.grid(column=0, row=2, columnspan=3, sticky="s", pady=5)

    def setup_port(self, parent, port_name, row):
        frame = ttk.LabelFrame(parent, text=port_name, padding="3")
        frame.grid(column=0, row=row, padx=5, pady=5, sticky=(tk.N, tk.S, tk.W, tk.E))
        status_label = ttk.Label(frame, text="Not Connected", font=("Cascadia Code", 10), foreground="red")
        status_label.grid(column=0, row=0, sticky=tk.W)
        text_widget = tk.Text(frame, width=28, height=6, wrap=tk.WORD)
        text_widget.grid(column=0, row=1, sticky=(tk.N, tk.S, tk.E, tk.W))
        self.port_widgets[port_name] = {'status_label': status_label, 'text_widget': text_widget}
        self.port_queues[port_name] = queue.Queue()

    def browse_folder(self):
        self.save_path = filedialog.askdirectory(title="Select Folder to Save Data")

    def check_connected_devices(self):
        device_mappings = get_device_mappings_by_usb_port()
        self.connected_ports = []
        for mapping in device_mappings:
            port_identifier = mapping['port_identifier']
            self.connected_ports.append(port_identifier)
            self.port_widgets[port_identifier]['status_label'].config(text="Connected", foreground="green")
        # Check for disconnected ports
        for port_name in self.port_widgets.keys():
            if port_name not in self.connected_ports:
                self.port_widgets[port_name]['status_label'].config(text="Not Connected", foreground="red")
        if not self.connected_ports and not self.logging_active:
            messagebox.showwarning("No Devices", "No FED3 devices are connected. Connect your devices.")

    def refresh_device_status(self):
        self.check_connected_devices()
        # Schedule the next check
        self.root.after(5000, self.refresh_device_status)

    def display_recording_indicator(self):
        if self.recording_circle is None:
            self.recording_circle = self.canvas.create_oval(10, 10, 50, 50, fill="red")
        if self.recording_label is None:
            self.recording_label = self.canvas.create_text(30, 60, text="RECORDING", font=("Cascadia Code", 10), anchor="n")

    def hide_recording_indicator(self):
        if self.recording_circle is not None:
            self.canvas.delete(self.recording_circle)
            self.recording_circle = None
        if self.recording_label is not None:
            self.canvas.delete(self.recording_label)
            self.recording_label = None

    def start_experiment(self):
        if not self.connected_ports:
            messagebox.showwarning("No Devices", "No FED3 devices are connected.")
            return
        self.logging_active = True
        self.experimenter_entry.config(state='disabled')
        self.experiment_entry.config(state='disabled')
        self.start_button.config(state='disabled')

        self.experimenter_name.set(self.experimenter_name.get().strip().lower())
        self.experiment_name.set(self.experiment_name.get().strip().lower())
        self.data_to_save = {port_identifier: [] for port_identifier in self.port_widgets.keys()}
        self.threads = []

        for mapping in get_device_mappings_by_usb_port():
            serial_port = mapping['serial_port']
            port_identifier = mapping['port_identifier']
            gpio_pins = gpio_pins_per_device.get(port_identifier)
            q = self.port_queues[port_identifier]
            status_label = self.port_widgets[port_identifier]['status_label']
            t = threading.Thread(target=read_from_fed, args=(serial_port, port_identifier, gpio_pins, q, status_label))
            t.daemon = True
            t.start()
            self.threads.append(t)
            self.serial_ports[port_identifier] = serial_port  # Store the serial port

        self.display_recording_indicator()

    def update_gui(self):
        for port_identifier, q in self.port_queues.items():
            try:
                while True:
                    message = q.get_nowait()
                    if isinstance(message, list):
                        self.data_to_save[port_identifier].append(message)
                    elif message == "Ready":
                        self.port_widgets[port_identifier]['status_label'].config(text="Connected", foreground="green")
                    else:
                        text_widget = self.port_widgets[port_identifier]['text_widget']
                        text_widget.insert(tk.END, message + "\n")
                        text_widget.see(tk.END)
            except queue.Empty:
                pass
        self.root.after(100, self.update_gui)

    def stop_experiment(self):
        if not self.logging_active:
            self.root.quit()
            self.root.destroy()
            return

        stop_event.set()
        for t in self.threads:
            t.join()
        GPIO.cleanup()
        self.save_all_data()
        self.hide_recording_indicator()
        self.logging_active = False
        self.root.quit()
        self.root.destroy()

    def save_all_data(self):
        if not self.save_path:
            messagebox.showerror("Error", "Please select a folder to save data.")
            return
        current_time = datetime.datetime.now().strftime("%Y_%m_%d_%H_%M_%S")
        experimenter_name = re.sub(r'[<>:"/\\|?*]', '_', self.experimenter_name.get())
        experiment_name = re.sub(r'[<>:"/\\|?*]', '_', self.experiment_name.get())

        experimenter_folder = os.path.join(self.save_path, experimenter_name)
        experiment_folder = os.path.join(experimenter_folder, f"{experiment_name}_{current_time}")
        os.makedirs(experiment_folder, exist_ok=True)
        for port_identifier, data_rows in self.data_to_save.items():
            if data_rows:
                filename_user = os.path.join(experiment_folder, f"{port_identifier}_{current_time}.csv")
                try:
                    with open(filename_user, mode='w', newline='') as file:
                        writer = csv.writer(file)
                        writer.writerow(column_headers)
                        writer.writerows(data_rows)
                    logging.info(f"Data saved for {port_identifier} in {filename_user}")
                except Exception as e:
                    logging.error(f"Failed to save data for {port_identifier}: {e}")
            else:
                logging.info(f"No data collected from {port_identifier}, no file saved.")
        messagebox.showinfo("Data Saved", "All data has been saved locally.")

    def on_closing(self):
        if self.logging_active:
            if messagebox.askokcancel("Quit", "Logging is active. Do you want to stop and exit?"):
                self.stop_experiment()
        else:
            self.root.quit()
            self.root.destroy()

if __name__ == "__main__":
    splash_root = tk.Tk()
    splash_screen = SplashScreen(splash_root)
    splash_root.mainloop()

    root = tk.Tk()
    app = FED3MonitorApp(root)
    root.mainloop()


# update dynamics

In [ ]:
#!/usr/bin/env python3

import os
import sys
import RPi.GPIO as GPIO
import serial
import threading
import datetime
import time
import logging
import re
import tkinter as tk
from tkinter import ttk, filedialog, messagebox
import queue
import csv
import webbrowser

# Configure logging
logging.basicConfig(
    level=logging.INFO,
    format='%(asctime)s [%(levelname)s] %(message)s',
    stream=sys.stdout
)

# Setup GPIO pins on the Raspberry Pi (BCM mode)
GPIO.setmode(GPIO.BCM)
GPIO.setwarnings(False)

# Define GPIO pins for each device
gpio_pins_per_device = {
    'Port 1': {"LeftPoke": 17, "RightPoke": 27, "Pellet": 22},
    'Port 2': {"LeftPoke": 10, "RightPoke": 9, "Pellet": 11},
    'Port 3': {"LeftPoke": 0, "RightPoke": 5, "Pellet": 6},
    'Port 4': {"LeftPoke": 13, "RightPoke": 19, "Pellet": 26},
}

# Set all pins as output and initially set them to LOW
for device_pins in gpio_pins_per_device.values():
    for pin in device_pins.values():
        GPIO.setup(pin, GPIO.OUT)
        GPIO.output(pin, GPIO.LOW)

# Define global threading and data storage variables
pellet_lock = threading.Lock()
pellet_in_well = {}
stop_event = threading.Event()
column_headers = [
    "Timestamp", "Temp", "Humidity", "Library_Version", "Session_type",
    "Device_Number", "Battery_Voltage", "Motor_Turns", "FR", "Event", "Active_Poke",
    "Left_Poke_Count", "Right_Poke_Count", "Pellet_Count", "Block_Pellet_Count",
    "Retrieval_Time", "InterPelletInterval", "Poke_Time"
]

def send_ttl_signal(pin):
    GPIO.output(pin, GPIO.HIGH)
    time.sleep(0.1)
    GPIO.output(pin, GPIO.LOW)

# Handle pellet events
def handle_pellet_event(event_type, port_identifier, gpio_pins, q):
    global pellet_in_well
    with pellet_lock:
        if port_identifier not in pellet_in_well:
            pellet_in_well[port_identifier] = False
        if event_type == "Pellet":
            if pellet_in_well[port_identifier]:
                GPIO.output(gpio_pins["Pellet"], GPIO.LOW)
                q.put(f"Pellet taken, signal turned OFF.")
                pellet_in_well[port_identifier] = False
                send_ttl_signal(gpio_pins["Pellet"])
            else:
                q.put(f"No pellet was in the well, no signal for pellet taken.")
        elif event_type == "PelletInWell":
            GPIO.output(gpio_pins["Pellet"], GPIO.HIGH)
            pellet_in_well[port_identifier] = True
            q.put(f"Pellet dispensed in well, signal ON.")

def process_event(event_type, port_identifier, gpio_pins, q):
    timestamp = datetime.datetime.now().strftime("%Y-%m-%d %H:%M:%S.%f")[:-3]
    message = f"[{timestamp}] {port_identifier} - Event: {event_type}"
    q.put(message)
    if event_type == "Left":
        send_ttl_signal(gpio_pins["LeftPoke"])
    elif event_type == "Right":
        send_ttl_signal(gpio_pins["RightPoke"])
    elif event_type == "LeftWithPellet":
        send_ttl_signal(gpio_pins["LeftPoke"])
    elif event_type == "RightWithPellet":
        send_ttl_signal(gpio_pins["RightPoke"])
    elif event_type in ["Pellet", "PelletInWell"]:
        handle_pellet_event(event_type, port_identifier, gpio_pins, q)

def get_device_mappings_by_usb_port():
    device_mappings = []
    usb_port_mapping = {'usb-0:1.1': 'Port 1', 'usb-0:1.2': 'Port 2', 'usb-0:1.3': 'Port 3', 'usb-0:1.4': 'Port 4'}
    by_path_dir = '/dev/serial/by-path/'
    if not os.path.exists(by_path_dir):
        return device_mappings  # Return empty list if directory doesn't exist
    for symlink in os.listdir(by_path_dir):
        symlink_path = os.path.join(by_path_dir, symlink)
        serial_port = os.path.realpath(symlink_path)
        if 'ttyACM' in serial_port or 'ttyUSB' in serial_port:
            usb_port_path = get_usb_port_path_from_symlink(symlink)
            if usb_port_path in usb_port_mapping:
                device_mappings.append({'serial_port': serial_port, 'port_identifier': usb_port_mapping[usb_port_path]})
    return device_mappings

def get_usb_port_path_from_symlink(symlink):
    match = re.search(r'usb-\d+:\d+(\.\d+)*', symlink)
    return match.group() if match else None

def read_from_fed(serial_port, port_identifier, gpio_pins, q, status_label):
    try:
        ser = serial.Serial(serial_port, 115200, timeout=1)
        q.put("Ready")
        status_label.config(text="Connected", foreground="green")
        while not stop_event.is_set():
            try:
                line = ser.readline().decode('utf-8').strip()
                if line:
                    data_list = line.split(",")
                    q.put(f"{port_identifier} raw data: {data_list}")
                    if len(data_list) >= 10:
                        event_type = data_list[9]
                        timestamp = datetime.datetime.now().strftime("%Y-%m-%d %H:%M:%S.%f")[:-3]
                        data_list[0] = timestamp
                        process_event(event_type, port_identifier, gpio_pins, q)
                        q.put(data_list)
            except serial.SerialException:
                q.put(f"Device on {port_identifier} disconnected.")
                status_label.config(text="Disconnected", foreground="red")
                break  # Exit loop if device is disconnected
    except serial.SerialException:
        q.put(f"Error opening serial port: {serial_port}")
        status_label.config(text="Disconnected", foreground="red")
    finally:
        if 'ser' in locals() and ser.is_open:
            ser.close()
        q.put(f"Stopped reading from {port_identifier}")

class SplashScreen:
    def __init__(self, root, duration=3000):
        self.root = root
        self.root.overrideredirect(True)
        self.root.geometry("800x480")
        self.root.attributes("-alpha", 1)

        self.root.configure(bg="black")
        self.label = tk.Label(
            self.root,
            text="McCutcheonLab Technologies",
            font=("Cascadia Code", 28, "bold"),
            bg="black",
            fg="lavender"
        )
        self.label.pack(expand=True)
        self.fade_in_out(duration)

    def fade_in_out(self, duration):
        self.fade_in(1000, lambda: self.fade_out(2000, self.close))

    def fade_in(self, time_ms, callback):
        alpha = 0.0
        increment = 1 / (time_ms // 50)

        def fade():
            nonlocal alpha
            if alpha < 1.0:
                alpha += increment
                self.root.attributes("-alpha", alpha)
                self.root.after(50, fade)
            else:
                callback()

        fade()

    def fade_out(self, time_ms, callback):
        alpha = 1.0
        decrement = 1 / (time_ms // 50)

        def fade():
            nonlocal alpha
            if alpha > 0.0:
                alpha -= decrement
                self.root.attributes("-alpha", alpha)
                self.root.after(50, fade)
            else:
                callback()

        fade()

    def close(self):
        self.root.destroy()

class FED3MonitorApp:

    def __init__(self, root):
        self.root = root
        self.root.title("HPFED TTL Monitor")
        self.root.geometry("800x480")
        self.root.attributes("-fullscreen", True)

        self.port_widgets = {}
        self.port_queues = {}
        self.experimenter_name = tk.StringVar()
        self.experiment_name = tk.StringVar()
        self.save_path = ""
        self.data_to_save = {}
        self.threads = []
        self.connected_ports = set()
        self.serial_ports = {}  # Store serial objects
        self.logging_active = False

        # Queue for log messages
        self.log_queue = queue.Queue()

        # Mainframe layout
        self.mainframe = ttk.Frame(self.root)
        self.mainframe.grid(column=0, row=0, sticky=(tk.N, tk.W, tk.E, tk.S))
        self.root.grid_rowconfigure(0, weight=1)
        self.root.grid_columnconfigure(0, weight=1)

        # Layout Setup
        self.create_layout()
        self.check_connected_devices()
        self.root.after(100, self.update_gui)
        self.root.after(5000, self.refresh_device_status)  # Periodically refresh device status
        self.root.protocol("WM_DELETE_WINDOW", self.on_closing)

    def create_layout(self):
        # Left side: Port 1 and Port 2
        left_ports = ttk.Frame(self.mainframe)
        left_ports.grid(column=0, row=0, sticky=(tk.N, tk.S, tk.W))

        self.setup_port(left_ports, 'Port 1', 0)
        self.setup_port(left_ports, 'Port 2', 1)

        # Right side: Port 3 and Port 4
        right_ports = ttk.Frame(self.mainframe)
        right_ports.grid(column=2, row=0, sticky=(tk.N, tk.S, tk.E))

        self.setup_port(right_ports, 'Port 3', 0)
        self.setup_port(right_ports, 'Port 4', 1)

        # Center controls
        controls_frame = ttk.Frame(self.mainframe)
        controls_frame.grid(column=1, row=0, padx=10, sticky=(tk.N, tk.S))

        tk.Label(controls_frame, text="Your Name:", font=("Cascadia Code", 8)).grid(column=0, row=0, sticky=tk.W)
        self.experimenter_entry = ttk.Entry(controls_frame, textvariable=self.experimenter_name, width=20)
        self.experimenter_entry.grid(column=1, row=0, sticky=tk.W)

        tk.Label(controls_frame, text="Experiment Name:", font=("Cascadia Code", 8)).grid(column=0, row=1, sticky=tk.W)
        self.experiment_entry = ttk.Entry(controls_frame, textvariable=self.experiment_name, width=20)
        self.experiment_entry.grid(column=1, row=1, sticky=tk.W)

        # Start and Stop buttons
        self.start_button = tk.Button(
            controls_frame,
            text="START",
            font=("Cascadia Code", 9, "bold"),
            bg="green",
            fg="white",
            command=self.start_experiment
        )
        self.start_button.grid(column=0, row=2, columnspan=2, sticky="we", pady=5)

        self.stop_button = tk.Button(
            controls_frame,
            text="STOP & SAVE",
            font=("Cascadia Code", 9, "bold"),
            bg="red",
            fg="white",
            command=self.stop_experiment
        )
        self.stop_button.grid(column=0, row=3, columnspan=2, sticky="we", pady=5)

        # Browse button
        self.browse_button = tk.Button(
            controls_frame,
            text="Browse Data Folder",
            font=("Cascadia Code", 9),
            command=self.browse_folder
        )
        self.browse_button.grid(column=0, row=4, columnspan=2, sticky="we", pady=5)

        # Recording Indicator
        self.canvas = tk.Canvas(controls_frame, width=120, height=100)
        self.canvas.grid(column=0, row=5, columnspan=2, pady=8)
        self.recording_circle = self.canvas.create_oval(10, 10, 50, 50, fill="red")
        self.recording_label = self.canvas.create_text(30, 60, text="OFF", font=("Cascadia Code", 10), anchor="n")

        # Log Section
        log_frame = ttk.Frame(self.mainframe)
        log_frame.grid(column=0, row=1, columnspan=3, pady=5, sticky=(tk.N, tk.S, tk.E, tk.W))

        self.log_text = tk.Text(log_frame, height=10, width=100, wrap=tk.WORD)
        self.log_text.pack(side=tk.LEFT, fill=tk.BOTH, expand=True)

        log_scrollbar = ttk.Scrollbar(
            log_frame,
            orient="vertical",
            command=self.log_text.yview
        )
        log_scrollbar.pack(side=tk.RIGHT, fill=tk.Y)
        self.log_text.configure(yscrollcommand=log_scrollbar.set)

        # Footer Frame for Copyright and Hyperlink
        footer_frame = ttk.Frame(self.mainframe)
        footer_frame.grid(column=0, row=2, columnspan=3, pady=5, sticky=(tk.S, tk.E, tk.W))

        # Updated Copyright
        copyright_label = tk.Label(
            footer_frame,
            text="© 2024 McCutcheonLab | UiT | Norway",
            font=("Cascadia Code", 8),
            fg="royalblue"
        )
        copyright_label.pack(pady=2)

        # Hyperlink
        hyperlink_label = tk.Label(
            footer_frame,
            text="Developed by Hamid Taghipourbibalan",
            font=("Cascadia Code", 8, "italic"),
            fg="blue",
            cursor="hand2"
        )
        hyperlink_label.pack(pady=2)
        hyperlink_label.bind("<Button-1>", lambda e: self.open_hyperlink("https://www.linkedin.com/in/hamid-taghipourbibalan-b7239088/"))

    def open_hyperlink(self, URL):
        webbrowser.open_new(URL)

    def setup_port(self, parent, port_name, row):
        frame = ttk.LabelFrame(parent, text=port_name, padding="3")
        frame.grid(column=0, row=row, padx=5, pady=5, sticky=(tk.N, tk.S, tk.W, tk.E))
        status_label = ttk.Label(frame, text="Not Connected", font=("Cascadia Code", 10), foreground="red")
        status_label.grid(column=0, row=0, sticky=tk.W)

        # Indicator Canvas
        indicator_canvas = tk.Canvas(frame, width=20, height=20)
        indicator_canvas.grid(column=1, row=0, padx=5)
        indicator_circle = indicator_canvas.create_oval(5, 5, 15, 15, fill="gray")

        # Text widget for port data
        text_widget = tk.Text(frame, width=28, height=6, wrap=tk.WORD)
        text_widget.grid(column=0, row=1, columnspan=2, sticky=(tk.N, tk.S, tk.E, tk.W))

        # Store widgets in the port_widgets dictionary
        self.port_widgets[port_name] = {
            'status_label': status_label,
            'text_widget': text_widget,
            'indicator_canvas': indicator_canvas,
            'indicator_circle': indicator_circle
        }
        self.port_queues[port_name] = queue.Queue()  # Initialize queue for each port

    def browse_folder(self):
        self.save_path = filedialog.askdirectory(title="Select Folder to Save Data")
        if self.save_path:
            self.log_queue.put(f"Data folder selected: {self.save_path}")
            logging.info(f"Save path selected: {self.save_path}")  # For debugging

    def check_connected_devices(self):
        device_mappings = get_device_mappings_by_usb_port()
        current_connected = set()
        for mapping in device_mappings:
            port_identifier = mapping['port_identifier']
            current_connected.add(port_identifier)
            if port_identifier not in self.connected_ports:
                # New device connected
                self.connected_ports.add(port_identifier)
                self.port_widgets[port_identifier]['status_label'].config(text="Connected", foreground="green")
                self.log_queue.put(f"{port_identifier} connected.")
                # Start logging thread for the new device if logging is active
                if self.logging_active:
                    self.start_logging_thread(mapping['serial_port'], port_identifier)
        # Check for disconnected devices
        disconnected = self.connected_ports - current_connected
        for port_identifier in disconnected:
            self.connected_ports.remove(port_identifier)
            self.port_widgets[port_identifier]['status_label'].config(text="Not Connected", foreground="red")
            self.log_queue.put(f"{port_identifier} disconnected.")
            # Stop logging thread if exists
            if port_identifier in self.serial_ports:
                # Signal the thread to stop
                stop_event.set()
                self.threads.remove(self.serial_ports[port_identifier]['thread'])
                # Close the serial port
                ser = self.serial_ports[port_identifier]['serial_port']
                if ser.is_open:
                    ser.close()
                del self.serial_ports[port_identifier]
        if not self.connected_ports and not self.logging_active:
            messagebox.showwarning("No Devices", "No FED3 devices are connected. Connect your devices.")

    def refresh_device_status(self):
        self.check_connected_devices()
        # Schedule the next check
        self.root.after(5000, self.refresh_device_status)

    def display_recording_indicator(self):
        self.canvas.itemconfig(self.recording_circle, fill="yellow")
        self.canvas.itemconfig(self.recording_label, text="RECORDING", fill="black")

    def hide_recording_indicator(self):
        self.canvas.itemconfig(self.recording_circle, fill="red")
        self.canvas.itemconfig(self.recording_label, text="OFF", fill="red")

    def start_experiment(self):
        if not self.connected_ports:
            messagebox.showwarning("No Devices", "No FED3 devices are connected.")
            return
        self.logging_active = True
        self.experimenter_entry.config(state='disabled')
        self.experiment_entry.config(state='disabled')
        self.start_button.config(state='disabled')

        # Normalize inputs
        self.experimenter_name.set(self.experimenter_name.get().strip().lower())
        self.experiment_name.set(self.experiment_name.get().strip().lower())

        # Validate input fields
        if not self.experimenter_name.get() or not self.experiment_name.get():
            messagebox.showerror("Error", "Please provide your name and experiment name.")
            return

        if not self.save_path:
            messagebox.showerror("Error", "Please select a data folder to save data.")
            return

        self.data_to_save = {port_identifier: [] for port_identifier in self.port_widgets.keys()}
        self.threads = []

        for mapping in get_device_mappings_by_usb_port():
            serial_port = mapping['serial_port']
            port_identifier = mapping['port_identifier']
            self.start_logging_thread(serial_port, port_identifier)

        self.display_recording_indicator()
        self.log_queue.put("Experiment started.")

    def start_logging_thread(self, serial_port, port_identifier):
        gpio_pins = gpio_pins_per_device.get(port_identifier)
        q = self.port_queues[port_identifier]
        status_label = self.port_widgets[port_identifier]['status_label']
        t = threading.Thread(target=read_from_fed, args=(serial_port, port_identifier, gpio_pins, q, status_label))
        t.daemon = True
        t.start()
        self.threads.append(t)
        self.serial_ports[port_identifier] = {'thread': t, 'serial_port': serial_port}
        self.log_queue.put(f"Started logging thread for {port_identifier}.")

    def update_gui(self):
        # Update port text widgets and handle indicators
        for port_identifier, q in self.port_queues.items():
            try:
                while True:
                    message = q.get_nowait()
                    if isinstance(message, list):
                        self.data_to_save[port_identifier].append(message)
                    elif message.startswith("Pellet"):
                        # Handle pellet events
                        text_widget = self.port_widgets[port_identifier]['text_widget']
                        text_widget.insert(tk.END, message + "\n")
                        text_widget.see(tk.END)
                    elif message == "Ready":
                        self.port_widgets[port_identifier]['status_label'].config(text="Connected", foreground="green")
                    elif "raw data" in message:
                        text_widget = self.port_widgets[port_identifier]['text_widget']
                        text_widget.insert(tk.END, message + "\n")
                        text_widget.see(tk.END)
                    elif message == "RIGHT_POKE":
                        self.trigger_indicator(port_identifier)
                    else:
                        text_widget = self.port_widgets[port_identifier]['text_widget']
                        text_widget.insert(tk.END, message + "\n")
                        text_widget.see(tk.END)
            except queue.Empty:
                pass

        # Process log messages
        try:
            while True:
                log_message = self.log_queue.get_nowait()
                timestamp = datetime.datetime.now().strftime("%m/%d/%Y %H:%M:%S")
                self.log_text.insert(tk.END, f"{timestamp}: {log_message}\n")
                self.log_text.see(tk.END)
        except queue.Empty:
            pass

        self.root.after(100, self.update_gui)

    def trigger_indicator(self, port_identifier):
        indicator_canvas = self.port_widgets[port_identifier]['indicator_canvas']
        indicator_circle = self.port_widgets[port_identifier]['indicator_circle']
        # Start blinking the indicator
        def blink(times):
            if times > 0:
                current_color = indicator_canvas.itemcget(indicator_circle, 'fill')
                next_color = 'red' if current_color == 'gray' else 'gray'
                indicator_canvas.itemconfig(indicator_circle, fill=next_color)
                self.root.after(250, lambda: blink(times -1))
            else:
                # Ensure the indicator is set back to 'gray' at the end
                indicator_canvas.itemconfig(indicator_circle, fill='gray')
        blink(6)  # Blink 6 times (3 cycles)

    def stop_experiment(self):
        if not self.logging_active:
            self.root.quit()
            self.root.destroy()
            return

        self.logging_active = False
        stop_event.set()
        self.hide_recording_indicator()
        self.log_queue.put("Stopping experiment...")

        # Wait for all threads to finish
        for port_identifier, info in self.serial_ports.items():
            t = info['thread']
            t.join(timeout=5)
            self.log_queue.put(f"Stopped logging thread for {port_identifier}.")

        # Save data
        self.save_all_data()

        # Reset for future experiments
        self.data_to_save = {}
        self.serial_ports = {}
        self.threads = []
        self.connected_ports = set()

        self.enable_input_fields()
        self.log_queue.put("Experiment stopped and data saved.")
        self.root.quit()
        self.root.destroy()

    def save_all_data(self):
        try:
            if not self.save_path:
                self.log_queue.put("Error: Save path is not set.")
                messagebox.showerror("Error", "Save path is not set.")
                return

            current_time = datetime.datetime.now().strftime("%Y_%m_%d_%H_%M_%S")
            experimenter_name = re.sub(r'[<>:"/\\|?*]', '_', self.experimenter_name.get())
            experiment_name = re.sub(r'[<>:"/\\|?*]', '_', self.experiment_name.get())

            experimenter_folder = os.path.join(self.save_path, experimenter_name)
            experiment_folder = os.path.join(experimenter_folder, f"{experiment_name}_{current_time}")
            os.makedirs(experiment_folder, exist_ok=True)

            self.log_queue.put(f"Experiment folder created at {experiment_folder}")

            for port_identifier, data_rows in self.data_to_save.items():
                if data_rows:  # Only save if there is data
                    filename_user = os.path.join(experiment_folder, f"{port_identifier}_{current_time}.csv")
                    try:
                        with open(filename_user, mode='w', newline='') as file:
                            writer = csv.writer(file)
                            writer.writerow(column_headers)
                            writer.writerows(data_rows)
                        logging.info(f"Data saved for {port_identifier} in {filename_user}")
                        self.log_queue.put(f"Data saved for {port_identifier} in {filename_user}.")
                    except Exception as e:
                        logging.error(f"Failed to save data for {port_identifier}: {e}")
                        self.log_queue.put(f"Failed to save data for {port_identifier}: {e}")
                else:
                    logging.info(f"No data collected from {port_identifier}, no file saved.")
                    self.log_queue.put(f"No data collected from {port_identifier}, no file saved.")
            messagebox.showinfo("Data Saved", "All data has been saved locally.")
        except Exception as e:
            logging.error(f"Error saving data: {e}")
            self.log_queue.put(f"Error saving data: {e}")
            messagebox.showerror("Error", f"Failed to save data: {e}")

    def enable_input_fields(self):
        self.experimenter_entry.config(state='normal')
        self.experiment_entry.config(state='normal')
        self.start_button.config(state='normal')

    def on_closing(self):
        if self.logging_active:
            if messagebox.askokcancel("Quit", "Logging is active. Do you want to stop and exit?"):
                self.stop_experiment()
        else:
            self.root.quit()
            self.root.destroy()

# Main execution
def main():
    # Splash screen
    splash_root = tk.Tk()
    splash_screen = SplashScreen(splash_root, duration=7000)  # 7 seconds duration
    splash_root.mainloop()

    # Main GUI
    root = tk.Tk()
    app = FED3MonitorApp(root)
    root.mainloop()

if __name__ == "__main__":
    main()
